In [66]:
import pandas as pd
import json

with open('word.json') as f:  
  data = json.load(f)


In [67]:
# delete key "vector" and add a new attribute "total_fre"
keys = ["0","1","2","3","4"]
time_len = 0
max_fre = 0
min_fre = 1
for i in keys:
    for j in data[i]:
        del data[i][j]["vevtor"]
        word_fre = sum(data[i][j]["time"])
        data[i][j]["total_fre"] = word_fre
        data[i][j]["cluster"] = i
        if(isinstance(data[i][j]["prediction"],list)):
            data[i][j]["prediction"] = data[i][j]["prediction"][0]
        elif(isinstance(data[i][j]["prediction"],float)):
            data[i][j]["prediction"] = data[i][j]["prediction"]
        if(word_fre>max_fre):
            max_fre = word_fre
        elif(word_fre<min_fre):
            min_fre = word_fre
        
        time_len = len(data[i][j]["time"])
print(data["0"]['abbott'])
print("max_fre:",max_fre)
print("min_fre:",min_fre)

{'time': [0.0, 0.0, 0.0, 5.64606052971009e-05, 9.767151117362088e-06, 3.913526713733348e-05, 3.4922488626192325e-05, 0.0, 0.0, 1.6468643213374495e-05, 2.7257432683522905e-05, 0.00013416806259056868, 9.569022232766968e-05, 2.700935174377141e-05, 1.478721201904593e-05, 2.050965424427371e-05, 8.628198706895538e-06, 1.4335364565650088e-05, 2.682273250777595e-05, 9.208009394478855e-06, 4.429531541760568e-06, 2.0507816111243362e-05, 2.2256676259335123e-05, 1.9118999518401845e-05, 1.0520527655899503e-05, 1.3013688969997028e-05, 1.0956372788330297e-05, 6.448941107301594e-06], 'prediction': 2.8790355653410958e-05, 'total_fre': 0.0006424229501272854, 'cluster': '0'}
max_fre: 0.19538808493064427
min_fre: 0.0


In [68]:
# re-normalize each time frequency, first filter out top 30 words for each time point, change all other values to 0
time_top_30_dict= {}
for i in range(time_len):
    time_top_30_dict[i] = []
    
for i in keys:
    for j in data[i]:
        for k in range(time_len):
            time_top_30_dict[k].append(data[i][j]["time"][k])
            
# sort each time_top_30_dict and keep only top 30s
for i in range(time_len):
    time_top_30_dict[i] = sorted(time_top_30_dict[i])[-30:]


In [69]:
time_top_30_dict

{0: [0.0028176773663983377,
  0.002828306438267013,
  0.0028303142773269175,
  0.0028501270161532762,
  0.0029359510592967147,
  0.003062316570126771,
  0.003092225954426834,
  0.0031167432093341593,
  0.0031752770720740162,
  0.0031812711968585744,
  0.0031827379928012415,
  0.003384936371399869,
  0.003480858104804677,
  0.003668318665863897,
  0.0037743642597467845,
  0.0038475848786136586,
  0.003894876221149016,
  0.003897773121693422,
  0.004445252104765782,
  0.004589996109408242,
  0.0045960208070666775,
  0.004757122101176693,
  0.005124293006148745,
  0.0054101299738815705,
  0.005418809939040764,
  0.0054245601262908815,
  0.006413451164233177,
  0.00710611021403884,
  0.0076145492516051195,
  0.011969181864673521],
 1: [0.00265366450845563,
  0.002754368994054808,
  0.0027959936620022526,
  0.0028448960310901014,
  0.002863331098124533,
  0.0029094039097855573,
  0.0031052984420715532,
  0.0031069152524653533,
  0.0031118942765025422,
  0.0031532572559332165,
  0.0032340527

In [70]:
# change all values not in top 30 to 0

for i in keys:
    for j in data[i]:
        for k in range(time_len):
            threshold = time_top_30_dict[k][0] 
            if(data[i][j]["time"][k] < threshold ):
                data[i][j]["time"][k] = 0

In [71]:
# output
with open("word_filtered.json", "w") as outfile:
    json.dump(data, outfile)

In [72]:
# generate top 100 frequent words

top_100_freq = []
top_100_pred = []
for i in keys:
    for j in data[i]:
        top_100_freq.append(data[i][j]["total_fre"])
        preditcion = data[i][j]["prediction"]
        if(isinstance(preditcion,list)):
            top_100_pred.append(data[i][j]["prediction"][0])
        elif(isinstance(preditcion,float)):
            top_100_pred.append(data[i][j]["prediction"])

top_100_freq  =  sorted(top_100_freq)[-100:]
top_100_thre = top_100_freq[0]

top_100_pred  =  sorted(top_100_pred)[-100:]
pred_max = top_100_pred[-1]
pred_min = top_100_pred[0]

print("pred_max",pred_max)
print("pred_min",pred_min)


pred_max 0.011798759682149523
pred_min 0.0014289310454713106


In [74]:
# set range for predictions 

num_class = 5 # super low, low, median, high ,super high
range_unit =  (pred_max - pred_min)/num_class

sl_thre = pred_min + range_unit
low_thre = pred_min + 2*range_unit
median_thre = pred_min + 3*range_unit
high_thre = pred_min + 4*range_unit
sh_thre = pred_min + 5*range_unit



top_100_data = []

for i in keys:
    for j in data[i]:
        sample = data[i][j]
        if(sample["total_fre"]>=top_100_thre):
            if(sample["prediction"]>=pred_min and sample["prediction"]<sl_thre):
                
                data_instance = {
                    "cluster":i,
                    "word": j,
                    "prediction":"super low"
                                }
            elif(sample["prediction"]>=sl_thre and sample["prediction"]<low_thre):
                data_instance = {
                    "cluster":i,
                    "word": j,
                    "prediction":"low"
                                }
            elif(sample["prediction"]>=low_thre and sample["prediction"]<median_thre):
                data_instance = {
                    "cluster":i,
                    "word": j,
                    "prediction":"median"
                                }
            elif(sample["prediction"]>=median_thre and sample["prediction"]<high_thre):
                data_instance = {
                    "cluster":i,
                    "word": j,
                    "prediction":"high"
                                }
            else:
                data_instance = {
                    "cluster":i,
                    "word": j,
                    "prediction":"super high"
                                }
                
            top_100_data.append(data_instance)
    


In [76]:
# top_100_data

In [77]:
with open("top_100_data.json", "w") as outfile:
    json.dump(top_100_data, outfile)